# Download YouTube Video's Audio

In [1]:
! pip install pytube -q

In [2]:
from pytube import YouTube

In [3]:
#VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo" #obama

In [4]:
VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ' #batman

In [5]:
#VIDEO_URL = 'https://youtu.be/qNJRGHk7sN8'

In [6]:
yt = YouTube(VIDEO_URL)

In [7]:
yt.streams \
  .filter(only_audio = True, file_extension = 'mp4') \
  .first() \
  .download(filename = 'ytaudio.mp4')

'e:\\code-clause\\VidSense\\ytaudio.mp4'

In [8]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 6.1-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-dxva2 --enable-d3d11va --enable-libvpl --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libt

# English ASR with HuggingSound

In [9]:
!pip install huggingsound -q

In [10]:
from huggingsound import SpeechRecognitionModel


e:\code-clause\VidSense\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
device

'cpu'

In [13]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)


12/14/2023 19:21:19 - INFO - huggingsound.speech_recognition.model - Loading model...


OUT OF MEMORY (OOM) error

# Audio Chunking

In [14]:
import librosa

In [15]:
input_file = 'ytaudio.wav'

In [16]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [17]:
import soundfile as sf


In [18]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

In [19]:
i

8

# Audio Transcription / ASR / Speech to Text

In [26]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'{a}.wav')

In [27]:
audio_path

['0.wav',
 '1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav']

In [28]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 9/9 [02:00<00:00, 13.37s/it]


In [29]:
full_transcript = ' '

In [30]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [31]:
len(full_transcript)

3091

# Text Summarization

In [32]:
from transformers import pipeline

In [33]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 1.80k/1.80k [00:00<00:00, 1.44MB/s]
e:\code-clause\VidSense\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\auuch\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activa

In [34]:
summarized_text = summarization(full_transcript)

In [35]:
summarized_text[0]['summary_text']

" The role of cat woman has been played by litedary access arears from rto kit and shell fifer . Batman is about etwo sides of drauma batman's born of trama andtis film maybes the ridler and tat is kind of the seed from which everything else grew ."

Text Chunking before Summarization

In [37]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

summarized_text

[' The role of cat woman has been played by litedary access arears from r',
 ' The key was really trying to trust that mt chose me for a reason realso i',
 ' "Pinguwin" was truly something spectacular so new onts very interesting do you',
 ' Le bit that that that made it interesting for me so yet ided lot of people']